In [2]:
import tensorflow as tf
from GAN import GAN
import numpy as np
import os
import matplotlib.pyplot as plt
import lmdb, cv2
%matplotlib inline

In [ ]:

def get_image_paths(main_path):
    return([os.path.join(main_path, img) for img in os.listdir(main_path)])

def get_batch(batch_size):
    global env, cursor
    imgs = np.zeros(shape=[batch_size, 256 , 256, 3])
    for i, (key, val) in enumerate(cursor):
        imgs[i, ...] = cv2.resize(cv2.imdecode(np.frombuffer(val, np.uint8), 1), (256, 256))
        if i==(batch_size-1):
            break
     
    if i < (batch_size-1):
        env.close()
        env = lmdb.open("E:\\bird", readonly=True)
        txn = env.begin(write=False)
        cursor = txn.cursor()
        imgs = get_batch(batch_size)
    else:
        imgs = (imgs/255. - 0.5)/0.5
    return(imgs)

# GAN JS divergence with disc input noise (anealing) w. mini-batch

In [ ]:
env = lmdb.open("E:\\bird", readonly=True)
txn = env.begin(write=False)
cursor = txn.cursor()
z_dim = 1024

tf.reset_default_graph()
gan = GAN(verbose=True)
layers = list()

ns = tf.train.exponential_decay(0.1, tf.Variable(0, trainable=False), 1000, 0.95, staircase=True)

G_layers = [[1024, [4, 4], (1, 1), "valid", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
            [512, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
            [256, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
            [128, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
            [64, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
            [32, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
            [3, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 0, "th", None, 0]]

D_layers = [[32, [4, 4], (2, 2), 'same', tf.random_normal_initializer(stddev=0.02), 0, 'lr', None, ns],
            [64, [4, 4], (2, 2), 'same', tf.random_normal_initializer(stddev=0.02), 1, 'lr', None, 0],
            [128, [4, 4], (2, 2), 'same', tf.random_normal_initializer(stddev=0.02), 1, 'lr', None, 0],
            [256, [4, 4], (2, 2), 'same', tf.random_normal_initializer(stddev=0.02), 1, 'lr', None, 0],
            [512, [4, 4], (2, 2), 'same', tf.random_normal_initializer(stddev=0.02), 1, 'lr', None, 0],
            [1024, [4, 4], (2, 2), 'same', tf.random_normal_initializer(stddev=0.02), 1, 'lr', [4, 5], 0],
            [1, [4, 4], (1, 1), 'valid', tf.random_normal_initializer(stddev=0.02), 0, 'l', None, 0]]


gargs = gan.build_graph(G_layers, D_layers, tf.placeholder(tf.float32, [None, 256, 256, 3]),
                       tf.placeholder(tf.float32, [None, 1, 1, z_dim]), True, 1.0, "RMS", 2e-4, True)

gan.train(40,  200000, 100, get_batch, "n", z_dim, True, 
          "summary\\GAN_RMS_256_with_noise_mini_sm_an_z", "Model\\GAN_RMS_256_with_noise_mini_sm_an_z", gargs)
env.close()